In [2]:
import pandas as pd
import numpy as np

from nltk import pos_tag, word_tokenize
import re

from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.gaussian_process import GaussianProcessClassifier
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

import nltk
import nltk as nlp
import string
import re

Using TensorFlow backend.


In [4]:
train = pd.read_csv('no_stopwords_combined.csv')
test = pd.read_csv('testnostopwords.csv')

train_copy = train.copy(deep = True)
test_copy = test.copy(deep = True)

In [5]:
train_copy.head(3)

,Unnamed: 0,Unnamed: 0.1,title,text,subject,date,target
0,0,0,Ramaphosa's ANC election win lifts South Afric...,johannesburg reuters south african banking sto...,worldnews,"December 19, 2017",1
1,1,1,VIGILANTE PIRATES INTERCEDE Where Government F...,like soldiers oden vigilante group reported fe...,Government News,"Apr 1, 2016",0
2,2,2,"SICK! DEMOCRAT ORGANIZER, Mayor DeBlasio Emplo...",last week huma abedin husband anthony weiner w...,left-news,"May 28, 2017",0


In [6]:
train_copy.isnull().sum()

Unnamed: 0        0
Unnamed: 0.1      0
title             0
text            718
subject           0
date              0
target            0
dtype: int64

### Starter Code EDA Result

- It is better to use the NaN rows, as all of them correspond to fake news
- One of the local rows in the dataset with null text is however labelled "true news"
- Pipeline doesn;t work with NaN values, so we'll replace NaN by 'Empty' string

In [7]:
#train_copy.dropna(subset = ['text'], inplace = True)

In [8]:
train_copy.fillna('Empty', inplace = True)

## Logistic Reg

In [9]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LogisticRegression())])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

accuracy: 52.49%


In [10]:
print(confusion_matrix(test_copy['target'],
                      pred))

[[131 247]
 [135 291]]


## Support Vector Classifier

In [11]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LinearSVC())])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

accuracy: 50.62%


In [12]:
print(confusion_matrix(test_copy['target'],
                      pred))

[[155 223]
 [174 252]]


## Multinomial Naive Bayes Classifier

In [13]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LinearSVC())])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

accuracy: 50.62%


In [14]:
print(confusion_matrix(test_copy['target'],
                      pred))

[[155 223]
 [174 252]]


## Gradient Boost Classifier

In [15]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', GradientBoostingClassifier(loss = 'deviance',
                                                   learning_rate = 0.01,
                                                   n_estimators = 10,
                                                   max_depth = 5,
                                                   random_state=42))])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

print(confusion_matrix(test_copy['target'],
                      pred))

accuracy: 48.51%
[[330  48]
 [366  60]]


## XGBoost Classifier

In [16]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', XGBClassifier(loss = 'deviance',
                                        learning_rate = 0.01,
                                        n_estimators = 10,
                                        max_depth = 5,
                                        random_state=2020))])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

print(confusion_matrix(test_copy['target'],
                      pred))

accuracy: 48.51%
[[330  48]
 [366  60]]


## Random Forest Classifier

In [17]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', RandomForestClassifier())])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

print(confusion_matrix(test_copy['target'],
                      pred))

accuracy: 52.61%
[[ 28 350]
 [ 31 395]]


# LSTM - Recurrent Neural Nets

In [18]:
X = train_copy['text']
Y = train_copy['target']

le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [19]:
max_words = 500
max_len = 1000

tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(X)

sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences, maxlen = max_len)

def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

model = RNN()

In [20]:
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(),
             metrics = ['accuracy'])

In [21]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 50)          25000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [22]:
model.fit(sequences_matrix, Y, batch_size = 256, epochs = 20,
         validation_split = 0.2)

C:\Users\Siraz\Anaconda3\envs\gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 35918 samples, validate on 8980 samples
Epoch 1/20
35918/35918 [==============================] - 214s 6ms/step - loss: 0.2201 - accuracy: 0.9211 - val_loss: 0.1218 - val_accuracy: 0.9596
Epoch 2/20
35918/35918 [==============================] - 210s 6ms/step - loss: 0.0701 - accuracy: 0.9790 - val_loss: 0.0186 - val_accuracy: 0.9961
Epoch 3/20
35918/35918 [==============================] - 211s 6ms/step - loss: 0.0297 - accuracy: 0.9928 - val_loss: 0.0116 - val_accuracy: 0.9979
Epoch 4/20
35918/35918 [==============================] - 216s 6ms/step - loss: 0.0413 - accuracy: 0.9890 - val_loss: 0.0224 - val_accuracy: 0.9967
Epoch 5/20
35918/35918 [==============================] - 209s 6ms/step - loss: 0.0326 - accuracy: 0.9912 - val_loss: 0.0620 - val_accuracy: 0.9976
Epoch 6/20
35918/35918 [==============================] - 244s 7ms/step - loss: 0.0378 - accuracy: 0.9911 - val_loss: 0.0108 - val_accuracy: 0.9976
Epoch 7/20
35918/35918 [==============================] - 266s 

In [23]:
test_sequences = tok.texts_to_sequences(test_copy['text'])
test_sequences_matrix = sequence.pad_sequences(test_sequences, 
                                               maxlen = max_len)

accr = model.evaluate(test_sequences_matrix, test_copy['target'])
print('Accuracy :{:0.2f}'.format(accr[1]))

804/804 [==============================] - 10s 13ms/step
Accuracy :0.48


In [24]:
# serialize model to JSON
model_json = model.to_json()
with open("base_model_LSTM_arc.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("base_model_LSTM_weights.h5")